# README
<p>This noteboook aims to pull from the data lake and answer the use case aggregations as outlined in the Section labeled "Questions"</p>


# Imports

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import Window


## Loading in Data

In [0]:
contname = 'team5-project2' #azure storage account container
#THIS WILL CHANGE BASED ON TEAM: 'team2-project2', 'team3-project2', etc.

storage_acct_name = '20230821desa'

client_id = 'de4ff859-02b1-4e2f-9d16-b578fa03df4f' #aka: app id

tenant_id = '33da9f3f-4c1a-4640-8ce1-3f63024aea1d' #aka: directory id

service_credential = dbutils.secrets.get(scope="databricks-app-kv",key="databricks-application")

gold_path = f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/GoldLayer'

In [0]:
action = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/factAction/part-*"))
action.createOrReplaceTempView('action')

In [0]:
timestamp = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/dimTimestamp/part-*"))
timestamp.createOrReplaceTempView('timestamp')

In [0]:
commit = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/dimCommits/part-*")).limit(100000)
commit.createOrReplaceTempView('commit')

In [0]:
linker_commits_action = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/linker_commits_action/part-*")).limit(100000)
linker_commits_action.createOrReplaceTempView('linker_commits_action')

In [0]:
linker_orgs = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/linker_org/part-*")).limit(100000)
linker_orgs.createOrReplaceTempView('linker_orgs')

In [0]:
pull_request = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/dimPullRequests/part-*")).limit(10000)
pull_request.createOrReplaceTempView('pull_request')

In [0]:
repo = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/dimRepos/part-*"))
repo.createOrReplaceTempView('repo')

In [0]:
repo_names = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/repo_names/part-*")).limit(100000)
repo_names.createOrReplaceTempView('repo_names')

In [0]:
org_names = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/dimOrganizations/part-*")).limit(100000)
orgs.createOrReplaceTempView('orgs')


In [0]:
user = (spark.read.format('parquet')
      .option("header","true")
      .option('inferColumnTypes', True)
      .load(f"{gold_path}/dimUser/part-*"))
user.createOrReplaceTempView('user')

# Questions

Once the data has been preprocessed in the silver layer, you have several aggregations that need to be done for the gold layer. Sparrow Analytics will load your gold layer into a Data Warehouse (and, eventually a BI tool) to provide insights into GitHub use patterns. Data loaded into the gold layer should adhere to a flat, star, or snowflake schema. You are tasked with the following aggregations: 

Data aggregated by type of GitHub event per hour 
PushEvent data aggregated by ref type – whether the commit is on the main branch 
Breakdown of events by type and number of commits per event 
User activity should be aggregated so that a filterable chart can be populated with breakdowns of user activity by week or month. 
Breakdown of activity by project – find a unique use case 
Challenge: Based on the commit messages – breakdown the events by language 
<ul>
  <li>Data aggregated by type of GitHub event per hour ?</li>
  <li>PushEvent data aggregated by ref type – whether the commit is on the main branch</li>
  <li>Breakdown of events by type and number of commits per event </li>
  <li>User activity should be aggregated so that a filterable chart can be populated with breakdowns of user activity by week or month. </li>
  <li>Breakdown of activity by project – find a unique use case </li>
  <li>Challenge: Based on the commit messages – breakdown the events by language </li>
</ul>

## Per Hour
<p>Group the number of Events per hour</P>

In [0]:
#answer
action.join(timestamp, action.time_id == timestamp.time_id).groupBy('year_created','day_created','hour_created','action_type').count().sort('hour_created',ascending=False).limit(10).display()

year_created,day_created,hour_created,action_type,count
2022,29,23,IssueCommentEvent,2
2022,19,23,PullRequestEvent,14
2022,24,23,IssuesEvent,2
2022,23,23,WatchEvent,8
2022,17,23,PullRequestReviewCommentEvent,1
2022,24,23,PullRequestEvent,11
2022,30,23,CommitCommentEvent,1
2022,4,23,WatchEvent,5
2022,12,23,CreateEvent,30
2022,13,23,DeleteEvent,2


## Main branch? (By Event Type)
<p>Determine whether or not the commit was on the main / master branch</p>

In [0]:
%sql
-- Grabs the pull_request_id and whether or not the base_name is on main from pull_request
-- Then joins action on pull_request_id.
-- Finally groups by the event (action)
SELECT a.action,
 count(t1.Bool) as Count_Per_Event_on_main
FROM (
  SELECT pull_request_id, 
    CASE WHEN pull_request_base_name LIKE '%master'  OR pull_request_base_name like '%main'
      THEN TRUE 
    ELSE FALSE
  END as Bool
  FROM pull_request
) t1
JOIN action a on t1.pull_request_id == a.action_pull_request_id
GROUP BY a.action

action,Count_Per_Event_on_main
closed,56
created,1675
reopened,4


## Breakdown of events by type and number of commits per event
<p>Counts the number of commits per event type</p>

In [0]:
#answer, they are all Push Events
commit_linked = commit.join(linker_commits_action,commit.commit_url == linker_commits_action.commit_url,'inner')
commit_linked.join(action, commit_linked.action_id == action.action_id, 'inner').groupBy('action_type').count().display()

action_type,count
PushEvent,23817936


## User activity should be aggregated so that a filterable chart can be populated with breakdowns of user activity by week or month

In [0]:
%sql
SELECT month_created, count(action.action_actor_id) FROM user JOIN action
ON user.user_id == action.action_actor_id
JOIN timestamp on timestamp.time_id == action.time_id
group by month_created

month_created,count(action_actor_id)
1,1479


In [0]:
%sql
SELECT number_of_week_created, count(action.action_actor_id) FROM user JOIN action
ON user.user_id == action.action_actor_id
JOIN timestamp on timestamp.time_id == action.time_id
group by number_of_week_created

number_of_week_created,count(action_actor_id)
1,340
52,58
3,335
5,48
4,371
2,327


In [0]:
%sql
SELECT month_created, number_of_week_created, count(t1.action_actor_id) 
FROM (user JOIN action
ON user.user_id == action.action_actor_id
JOIN timestamp on timestamp.time_id == action.time_id) t1
group by number_of_week_created, month_created


month_created,number_of_week_created,count(action_actor_id)
1,3,335
1,1,340
1,2,327
1,52,58
1,5,48
1,4,371


Databricks visualization. Run in Databricks to view.

## Breakdown of activity by project – find a unique use case

In [0]:
#How many actions of each type for each repo_url
action.join(repo,action.action_repo_id == repo.repo_id,'inner').groupBy('repo_url','action_type').count().sort('repo_url').withColumnRenamed('count','Total Actions').limit(10).display()

repo_url,action_type,Total Actions
https://api.github.com/repos/0xStefan214/javascript-files,PushEvent,1
https://api.github.com/repos/0xsequence/erc-1155,WatchEvent,1
https://api.github.com/repos/117010130/hello-world,PushEvent,1
https://api.github.com/repos/1Password/scim-examples,PullRequestReviewEvent,1
https://api.github.com/repos/4vent/ArcadeTools,PushEvent,1
https://api.github.com/repos/8fn/exercicios-php,PublicEvent,1
https://api.github.com/repos/ABHINAV-GOPINADH/java-expt-2,CreateEvent,1
https://api.github.com/repos/AIAcademyBatchH/TensorFlow_ANN,PushEvent,1
https://api.github.com/repos/AIT-LAHCEN/Extranet_RH,CreateEvent,1
https://api.github.com/repos/Abh1shekSingh/Coursera_Solution,CreateEvent,1


In [0]:
#Repos with two or more actions
action.join(repo,action.action_repo_id == repo.repo_id,'inner').groupBy('repo_url').count().sort('repo_url').filter('count > 1').withColumnRenamed('count','Total Actions').limit(10).display()

repo_url,Total Actions
https://api.github.com/repos/BornToBeRoot/NETworkManager,2
https://api.github.com/repos/CSeanXu/b1llion,6
https://api.github.com/repos/CaeCur/lights-out,2
https://api.github.com/repos/Codecademy/gamut,4
https://api.github.com/repos/DataDog/documentation,4
https://api.github.com/repos/DimensionDev/Maskbook,3
https://api.github.com/repos/DonQuixoteJoker/Wabo,2
https://api.github.com/repos/EthereansOS/ITEMS-swap,2
https://api.github.com/repos/GuruCICDCanary-Prod-Release/CICDCanary,13
https://api.github.com/repos/Jason2866/tmp_copy,2


In [0]:
#Actions per month for each repo_url
resault = action.join(repo,action.action_repo_id == repo.repo_id,'inner')
resault.join(timestamp, resault.time_id == timestamp.time_id,'inner').groupBy('repo_url','day_created').count().sort('repo_url','day_created').limit(10).display()

repo_url,day_created,count
https://api.github.com/repos/0xStefan214/javascript-files,12,1
https://api.github.com/repos/0xsequence/erc-1155,24,1
https://api.github.com/repos/117010130/hello-world,17,1
https://api.github.com/repos/1Password/scim-examples,19,1
https://api.github.com/repos/4vent/ArcadeTools,31,1
https://api.github.com/repos/8fn/exercicios-php,17,1
https://api.github.com/repos/ABHINAV-GOPINADH/java-expt-2,25,1
https://api.github.com/repos/AIAcademyBatchH/TensorFlow_ANN,28,1
https://api.github.com/repos/AIT-LAHCEN/Extranet_RH,14,1
https://api.github.com/repos/Abh1shekSingh/Coursera_Solution,21,1


## More Queries

### Orgs!

In [0]:
orgs = linker_orgs.join(org_names,linker_orgs.org_logins_id == org_names.org_logins_id,'inner').drop('org_logins_id')

In [0]:
#number of total actions by an org
orgs.join(action,orgs.org_id == action.action_org_id,'inner').groupBy('org_login').count().sort('count',ascending=False).limit(10).display()

org_login,count
PlaNFT,82437
Fluffy-Pan,61364
MicrosoftDocs,53613
AzureSDKAutomation,52371
JetBrains,52243
Tencent,20968
MOVIECORD,20345
SAP,19170
IBM,17121
UBC-MDS,15773


In [0]:
#All Orgs Actions per day
join_1 = orgs.join(action,orgs.org_id == action.action_org_id,'inner')
join_1.join(timestamp, join_1.time_id == timestamp.time_id,'inner').groupBy('day_created').count().sort('day_created').limit(10).display()

day_created,count
1,40416
2,48507
3,89964
4,98923
5,110303
6,101268
7,98812
8,61855
9,58261
10,111763


### Users and Contributors

In [0]:
action_repo = action.join(repo,action.action_repo_id == repo.repo_id,'inner').groupBy('repo_url','action_actor_id').count().sort('repo_url',col('count').desc())
action_repo = action_repo.join(user,action_repo.action_actor_id==user.user_id,'inner')
window = Window.partitionBy(action_repo['repo_url']).orderBy(action_repo['count'].desc())

In [0]:
# Every contributer to a repo ranked by number of actions
contributors_ranked = action_repo.select('*', rank().over(window).alias('rank')).drop('action_actor_id')
contributors_ranked.limit(15).display()
   


repo_url,count,user_id,user_login,rank
https://api.github.com/repos/0-000/nobelium,1,60861062,0-000,1
https://api.github.com/repos/0-5d/stock_picker,2,51177522,0-5d,1
https://api.github.com/repos/0-6-1-7/rosreestr,1,96841762,Waht-is-it,1
https://api.github.com/repos/0-Azure/0-Azure,2,93076002,0-Azure,1
https://api.github.com/repos/0-Raigi-0/Cesar,2,72525501,0-Raigi-0,1
https://api.github.com/repos/0-Shine-0/Goodanimemes-Rules,1,62271116,worldeva,1
https://api.github.com/repos/0-TelitsynNikite/Mel,12,77964273,0-TelitsynNikite,1
https://api.github.com/repos/0-X0deX-o/TOP,6,91225758,0-X0deX-o,1
https://api.github.com/repos/0-ng/end_nginx,2,49059247,0-ng,1
https://api.github.com/repos/0-vortex/open-sauced-supabase-cron-test,1402,41898282,github-actions[bot],1


In [0]:
# The top contributer to every repo
top_contributor = action_repo.select('*', rank().over(window).alias('rank')).filter(col('rank') == 1)
top_contributor.limit(10).display()

repo_url,count,user_id,user_login,rank
https://api.github.com/repos/0-2HERO/FE21-CR01-Aleksandar,1,64213996,0-2HERO,1
https://api.github.com/repos/0-Dark-0/My-first-simple-calculator,2,98577400,0-Dark-0,1
https://api.github.com/repos/0-Diogo-Marques-0/0-Diogo-Marques-0,7,98153085,0-Diogo-Marques-0,1
https://api.github.com/repos/0-Rafinha/ocean-master,2,92826789,0-Rafinha,1
https://api.github.com/repos/0-a-e/OSarchive-UI,2,49699333,dependabot[bot],1
https://api.github.com/repos/0-ayano/OSandCompiler_1,26,86472869,0-ayano,1
https://api.github.com/repos/0-harshit-0/maze,31,59227793,0-harshit-0,1
https://api.github.com/repos/0-jagadeesh-0/food-order-app,3,74336885,0-jagadeesh-0,1
https://api.github.com/repos/0-shubham-0/EverKnight,3,89774595,0-shubham-0,1
https://api.github.com/repos/0-woo/0-woo.github.io,2,90828359,0-woo,1


In [0]:

tip_top_contributor = top_contributor.drop('count').groupBy('user_id').count().sort('count',ascending=False)

In [0]:
#Top contributors by number of repos they are the biggest contributor in
tip_top_contributor.join(user,tip_top_contributor.user_id==user.user_id,'left').sort('count',ascending=False).drop('user_id').limit(10).display()

count,user_login
1476195,dependabot[bot]
102861,direwolf-github
49077,github-actions[bot]
34996,pull[bot]
21791,berolinux
18454,vercel[bot]
18328,aws-aemilia
12031,renovate[bot]
9629,github-classroom[bot]
9619,Butters3388214


## Attempting to set up for NLP

In [0]:
!pip install spark-nlp

  Using cached spark_nlp-5.1.2-py2.py3-none-any.whl (536 kB)

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [0]:
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import LanguageDetectorDL
from sparknlp.base import *
# Does not work due to needing to install a JAR cluster wide
# This can be found several stack overflow discussions.
# The following is a possible solution, however it did not work for us
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")  # Will take your column name and output a diff name